# AudioMoth Data Preparation

This notebook prepares the dataset given by the wildlife trust.   
Aim is to confirm the data is as expected and prepare for future analysis. 

This notebook:
- Loads raw AudioMoth detections and device metadata
- Normalises and flattens dataset
- Validates and filters dataset
- Adds temporal features (hour, week, month)
- Outputs a single analysis-ready dataframe

Downstream notebooks assume this output is correct and stable.
# Setup Project Root

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd


# Go up one level to .../audiomoth
PROJECT_ROOT = Path(os.getcwd()).resolve().parent

# Add project root to sys.path so `src` is importable
sys.path.insert(0, str(PROJECT_ROOT))

EXCEL_PATH = PROJECT_ROOT / "data_raw" / "helman_tor_audiomoth_data.xlsx"

# Make pandas show more columns/rows while exploring
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

## Import Excel Data

In [ ]:
import src.data_store as data_store

# Get all the excel sheets available in the audiomoth data
sheets = data_store.get_excel_sheets(EXCEL_PATH)

## Basic Normalisation

In [ ]:
import src.normaliser as normaliser
import src.audio_moth_schema as audio_moth_schema

# Before merging we should combine date and time columns in Overview sheet
sheets["Overview"] = normaliser.combine_date_and_time(
    sheets["Overview"],
    date_col="deployment_date",
    time_col="deployment_time",
    output_col="deployment_timestamp",
)

# Combine date and time columns in All Data sheet
sheets["All Data"] = normaliser.combine_date_and_time(
    sheets["All Data"],
    date_col="date",
    time_col="time",
    output_col="detection_timestamp",
)

# Flatten all the sheets into a single DataFrame
df = normaliser.flatten_data(sheets)

## Validate Data

In [ ]:
# Validate, filter and convert types according to AudioMoth schema.
df = audio_moth_schema.AudioMothSchema.validate(df)

print(df.shape)
df.head()

## Add Temporal features.

In [ ]:
df["date"] = df["detection_timestamp"].dt.date  # type: ignore[reportAttributeAccessIssue]
df["hour"] = df["detection_timestamp"].dt.hour  # type: ignore[reportAttributeAccessIssue]
df["week"] = df["detection_timestamp"].dt.isocalendar().week  # type: ignore[reportAttributeAccessIssue]
df["month"] = df["detection_timestamp"].dt.to_period("M").astype(str)  # type: ignore[reportAttributeAccessIssue]

df.head()

## Brief data checks

In [ ]:
df.info()
df.describe(include="all")

In [ ]:
df["detection_timestamp"].min(), df["detection_timestamp"].max()

In [ ]:
df.assign(year=df["detection_timestamp"].dt.year)[
    ["year", "month", "week"]
].drop_duplicates().sort_values(["year", "week"])

## Save Prepared Data

In [ ]:
from pathlib import Path

out_dir = Path(PROJECT_ROOT) / "data_processed"
out_dir.mkdir(parents=True, exist_ok=True)

df.to_parquet(out_dir / "analysis_df.parquet", index=False)